In [29]:
import os 
import sys

In [30]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Boston-Regression\\Boston_Housing-Regression'

In [31]:
os.chdir('d:\\iNeuron\\Complete Project\\Boston-Regression\\Boston_Housing-Regression')

In [32]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Boston-Regression\\Boston_Housing-Regression'

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
     preprocessed_obj_file_path: Path
     scaled_train_path : Path
     scaled_test_path : Path

In [34]:
from src.utils import create_directories, read_yaml
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.logger import logging

class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        try:
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            create_directories([self.config.artifacts_root])
        except Exception as e:
            raise e
        
    def get_data_transformation_config(self):
        try:
            logging.info('Data Transformation config started.')
            config = self.config.data_transformation

            preprocessed_obj_file_path=config.preprocessed_obj_file_path
            scaled_train = config.scaled_train_path
            scaled_test = config.scaled_test_path

            data_transforamtion_config = DataTransformationConfig(
                preprocessed_obj_file_path=preprocessed_obj_file_path,
                scaled_train_path=scaled_train,
                scaled_test_path=scaled_test
            )

            return data_transforamtion_config
        except Exception as e:
            raise e 

In [35]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

from src.utils import save_obj


train_path = r'Boston_Housing-Regression\artifact\data_ingestion\train.csv'
test_path = r'Boston_Housing-Regression\artifact\data_ingestion\test.csv'

final_train_path = os.path.join(os.path.dirname(os.getcwd()), train_path)
final_test_path = os.path.join(os.path.dirname(os.getcwd()), test_path)


class DataTransformation:
    def __init__(self, config: DataTransformationConfig, train_path, test_path):
        self.config = config
        train_path = train_path
        test_path=test_path

    def initiate_data_transformation(self):
        logging.info('Initiating Data Transformation')
        train_df = pd.read_csv(final_train_path)
        test_df = pd.read_csv(final_test_path)

        train_target = train_df['MEDV']
        test_target = test_df['MEDV']
        train_df = train_df.drop('MEDV', axis=1)
        test_df = test_df.drop('MEDV', axis=1)
        
        scaler = StandardScaler()
        scaled_train_df = pd.DataFrame(scaler.fit_transform(train_df), columns = train_df.columns)
        scaled_test_df = pd.DataFrame(scaler.transform(test_df), columns = test_df.columns)

        final_train_df=pd.concat([scaled_train_df,train_target], axis=1)
        final_test_df=pd.concat([scaled_test_df,test_target], axis=1)

        os.makedirs(os.path.dirname(self.config.preprocessed_obj_file_path), exist_ok=True)

        save_obj(self.config.preprocessed_obj_file_path, scaled_test_df)

        final_train_df.to_csv(self.config.scaled_train_path, index=False, header=True)
        final_test_df.to_csv(self.config.scaled_test_path, index=False, header=True)
        logging.info('Train and test set has been preprocessed')

        logging.info('Data Transformation completed')
        logging.info('\n')
        return self.config.scaled_train_path, self.config.scaled_test_path

In [36]:
train_path = r'Boston_Housing-Regression\artifact\data_ingestion\train.csv'
test_path = r'Boston_Housing-Regression\artifact\data_ingestion\test.csv'

final_train_path = os.path.join(os.path.dirname(os.getcwd()), train_path)
final_test_path = os.path.join(os.path.dirname(os.getcwd()), test_path)

if __name__=='__main__':
    config=ConfigurationManager()
    data_transforamtion_config=config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transforamtion_config, train_path=final_train_path, test_path=final_test_path)
    data_transformation.initiate_data_transformation()

In [37]:
import os
print(os.environ.get('PYTHONPATH'))

D:\iNeuron\Complete Project\Boston-Regression\Boston_Housing-Regression\src


In [38]:
import os
os.environ['PYTHONPATH'] = 'D:\iNeuron\Complete Project\Boston-Regression\Boston_Housing-Regression\src'

In [39]:
import os
print(os.environ.get('PYTHONPATH'))

D:\iNeuron\Complete Project\Boston-Regression\Boston_Housing-Regression\src
